In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# if you mount Google drive correctly, the following commands should be able to be executed correctly
!ls /content/drive/
%cd "/content/drive/Shared drives/computer vision 2019"

'My Drive'  'Shared drives'
/content/drive/Shared drives/computer vision 2019


In [0]:
import time
import cv2
import torch 
import argparse
import numpy as np
import os 
import glob
import torch.nn.functional as F
import matplotlib.pyplot as plt


model = './SHM/ckpt/human_matting/model/model_obj.pth'
#model = 'model_obj.pth'
results = './SHM/results'
img_path = './SHM/test'
size = 256
without_gpu = False

torch.set_grad_enabled(False)

    
#################################
#----------------
if without_gpu:
    print("use CPU !")
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        n_gpu = torch.cuda.device_count()
        print("----------------------------------------------------------")
        print("|       use GPU !      ||   Available GPU number is {} !  |".format(n_gpu))
        print("----------------------------------------------------------")

        device = torch.device('cuda:0,1')

#################################
#---------------


In [0]:
def load_model(model, without_gpu):
    print('Loading model from {}...'.format(model))
    if without_gpu:
        myModel = torch.load(model, map_location=lambda storage, loc: storage)
    else:
        myModel = torch.load(model)

    myModel.eval()
    myModel.to(device)
    
    return myModel

def seg_process(size, without_gpu, image, net):

    # opencv
    origin_h, origin_w, c = image.shape
    image_resize = cv2.resize(image, (size,size), interpolation=cv2.INTER_CUBIC)
    image_resize = (image_resize - (104., 112., 121.,)) / 255.0



    tensor_4D = torch.FloatTensor(1, 3, size, size)
    
    tensor_4D[0,:,:,:] = torch.FloatTensor(image_resize.transpose(2,0,1))
    inputs = tensor_4D.to(device)

    t0 = time.time()

    trimap, alpha = net(inputs)

    print((time.time() - t0))  

    if without_gpu:
        alpha_np = alpha[0,0,:,:].data.numpy()
    else:
        alpha_np = alpha[0,0,:,:].cpu().data.numpy()


    alpha_np = cv2.resize(alpha_np, (origin_w, origin_h), interpolation=cv2.INTER_CUBIC)

    fg = np.multiply(alpha_np[..., np.newaxis], image)

    bg = image
    bg_gray = np.multiply(1-alpha_np[..., np.newaxis], image)
    bg_gray = cv2.cvtColor(bg_gray, cv2.COLOR_BGR2GRAY)

    bg[:,:,0] = bg_gray
    bg[:,:,1] = bg_gray
    bg[:,:,2] = bg_gray

    # fg[fg<=0] = 0
    # fg[fg>255] = 255
    # fg = fg.astype(np.uint8)
    # out = cv2.addWeighted(fg, 0.7, bg, 0.3, 0)
    out = fg# + bg
    out[out<0] = 0
    out[out>255] = 255
    out = out.astype(np.uint8)

    return out


def img_seg(size, without_gpu, results, img_path, model):
    for idx, img_name in enumerate(glob.glob(img_path+'/*')):
        img = cv2.imread(img_name)
        img_name = img_name.replace(img_path, '')
        img_out = seg_process(size, without_gpu, img, model)
        cv2.imwrite(results + img_name, img_out)




In [6]:
myModel = load_model(model, without_gpu)
img_seg(size, without_gpu, results, img_path, myModel)

Loading model from ./SHM/ckpt/human_matting/model/model_obj.pth...


ModuleNotFoundError: ignored

In [7]:
model

'./SHM/ckpt/human_matting/model/model_obj.pth'